In [ ]:
#(0) Graphics... import unicodedata
# (0.1) Error handling for a few numerical inputs, currently required to be integers
import copy

#(1) Create Card as object (no inputs needed)-----------------------------------------------------------------------
hand_score = 0

class Card(object):
       
    def __init__(self, rank=None, suit=None, rankval=None):
        #suits = ('Spades', 'Hearts', 'Clubs', 'Diamonds')
        #ranks = range(1, 14)
        
        #if rank not in ranks:
        #    raise TypeError('rank must be int 1-13')
            
        #if suit not in suits:
        #    raise TypeEror("suit must be 'Spades' etc")
        
        if rank:
            self.rank = rank
        else:
            self.rank = None
        if suit:
            self.suit = suit
        else:
            self.suit = None
        self.low = False             # Aces low (hidden attr.) set to false by default, below do bust check then change/recheck
        if rank in range(1,10):      # 1-9
            self.rankval = rank      # this attribute will be used in sums, winning etc; +1 coz num starts at 0
        else:
            self.rankval = 10
            
        # methods for modifying rank and suit, eg to generate Deck
    def new_rank(self, rank):
        self.rank = rank
            
    def new_suit(self, suit):
        self.suit = suit
        
    def new_rankval(self, rank):      # Delete??? At best, come back to this +++++++++++++++++++++++++++++++++
        if rank in range(0,11):       # 1-10, which I use below
            self.rankval = rank      # this attribute will be used in sums, winning etc; +1 coz num starts at 0
        elif rank=='Ace':
            try:
                self.rankval = 11 and handscore<21
            except:
                pass
   
    def __repr__(self):                                     # represent card in intuitive way (returns tuple)
        return "%s of %s" % (self.rank, self.suit)
    
    def highlow(self):                                      # converts to Aces low (if applicable)
        self.low=True
    
    def __int__(self):                                      # returns Blackjack-expected value of a card
        if type(self.rank) == int:                          # NB later get rid of rankval attribute
            return self.rank
        elif self.rank in ["King", "Queen", "Jack"]:
            return 10
        elif self.low:                                      # Aces Low business
            return 1
        else:
            return 11
        
    def __add__(self,other):                                # uses above value to simply add cards
        return int(self) + int(other)
    def __radd__(self,other):
        return self + other
            
                
            
#(2) Create Deck as object which self-populates and shuffles-----------------------------------------------------------
# with attribute called cards (name.cards)
# method to shuffle and take_1

class Deck(object):
    
    def __init__(self, cards=[], outgoing=[], decksize=1):
        self.cards = cards
        self.outgoing = outgoing
        self.decksize = decksize
        self.populate(decksize)
        
    def shuffle(self):
        from random import shuffle
        shuffle(self.cards)
        
    # Method to mudar last card (without replacement)
    def take_1(self): # later (self, n) to take n cards
        self.outgoing=self.cards[len(self.cards)-1]         # this is the removed (dealt) card
        self.cards.remove(self.cards[len(self.cards)-1])
        return self.outgoing
        
    #print(ranks)
    #print(suits)
    
# no don't #(2.1) Instantiate deck to later populate the instantiation (uses Card as object)
    
    def populate(self, decksize):
        
        self.cards = [] #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ NEW
        
        # Define suits and ranks for generating the members of the deck with double loop
        suits = ['Spades', 'Hearts', 'Diamonds', 'Clubs']
        ranks = range(1,14) # 1-13 but starts at index zero
        ranks = list(ranks)

        # Modify ranks to replace 1 with Ace, 11 with Jack, 12->Queen, 13->King
        ranks[0] = 'Ace'
        ranks[10] = 'Jack'
        ranks[11] = 'Queen'
        ranks[12] = 'King'

        # Allow multiple decks to be compounded (shuffle at very end)
        decksizedummy = 0
        while decksizedummy < decksize:
        
            # Apply double loop to populate deck with members
            for suit in suits:
                for rank in ranks:
                    # my_deck = Deck()
                    dummy_card = Card()
                    dummy_card.new_rank(rank)
                    dummy_card.new_rankval(rank)
                    dummy_card.new_suit(suit)
                    self.cards.append( dummy_card ) # apply .append method to list my_deck.cards
            decksizedummy += 1
        
        self.shuffle()
        
        #print(dummy_card)# print(id(dummy_card))
        # optional print out the cards as reality check
        # my_deck.cards        

        #(2.1.1) Reality check
        
        #print(self.cards[0:5])
        print("Deck created of size: " + str(len(self.cards)))

        



#(3) Create Player and Dealer (later make multi-player)-------------------------------------------------------------------

class Player(object):
# A Player has pot (default 1000), betting_box (default 0), place_bet (move from betting_box to pot)

    def __init__(self, pot=1000, betting_box=0, name='', hand = None, dealer=False):
        self.pot = pot # each player has money in game; to call this, don't need ()
        self.betting_box = betting_box # each player takes from pot to bet in this box, don't need ()
        if hand:
            self.hand = hand # the list of cards
        else:
            self.hand = []
        self.name = name # annoyingly, needs to be typed in twice
        self.dealer = dealer

    def place_bet(self, amount):
        self.betting_box += int(amount)
        self.pot -= int(amount) # to call needs amount because no default specified

    def win_BJ(self): # to win by Blackjack at payout 3:2 - needs ()
        self.pot += 2.5 * self.betting_box
        self.betting_box=0

    def win(self): # to win normally (1:1), needs ()
        self.pot += 2*self.betting_box
        self.betting_box=0

    def tie(self):
        self.pot += self.betting_box
        self.betting_box=0

    def lose(self):
        self.betting_box=0

    def receive(self, card): # to receive a card and add it to hand
        self.hand.append(card)

    def chuck(self): # method to discard cards (not stored because after this pack is renewed)
        self.hand=[]

    def __repr__(self):
        return (self.name)



#numdecks = input("How many decks (<9) ? ")
y=0
while y==0:
    try:
        numdecks = int(input("How many decks (<9) ? "))
        y=1
    except:
        print ('Must be number: 1,2,3,...,8')
        #numdecks = input("How many decks (<9) ? ")
numdecks = int(numdecks)

while 1>int(numdecks) or int(numdecks)>8:
    print ('Must be in range: 1,2,3,...,8')
    numdecks = input("How many decks (<9) ? ")
numdecks = int(numdecks)

deckdummy = 0
#my_deck_temp = Deck()
#while deckdummy<numdecks:
my_deck = Deck(decksize=numdecks)
    #my_deck_temp.cards.append(my_deck.cards) ### Can I do this? To enable multiple decks no that I've removed cumulativity
    #deckdummy += 1
#my_deck = my_deck_temp # Follows up
#print ("Total deck size: " + str(len(my_deck.cards))) # more follow-up
    
#(3.1.1) Create Player List and populate, create Dealer; up to 6 users input names and bankrolls---------------------------
playerlist=[]
while len(playerlist)<6: # allow up to 6 players
    #playerlist.append(input("What's your name, player? ")=Player())    # doesn't cut it
        x=input("Add player? (Y/N) ")
        if x == 'N' or x == 'n':
            break
        elif x in ['Y', 'y', 'yes']:
            playerlist.append(Player(name=input("What's your name, Player? "))) # already these Player() objects
            continue
        else:### test
            break # if this works, make concise

for player in playerlist:
    dummy1 = 0
    while dummy1 == 0:
        try:
            player.pot=int(input(player.name + ' : How much do you have in the bank? '))
            dummy1 = 1
        except ValueError:
            print ("Whole number geeze ")

Dealer = Player(pot=1000000, name='Dealer', dealer=True)     # later make this 1000*max(player.pot for player in playerlist)
#for player in playerlist:                                          # unnecessary here but may be nice later
 #   print(player.name + ' brings ' + player.pot)
playerlist.append(Dealer)          #  changed my mind about this twice, NB if dealer=false etc below
initial_playerlist=playerlist


# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# indefinite gaming loop from here-------------------------------------------------------------------------------
keepplaying=1
while keepplaying==1:

    
    # (4) Take initial bet-------------------------------------------------------------------------------------------------------
    #Player_1.place_bet(int(input('How much to play? ')))
    #print ('At stake for %s: %s; pot: %s' %('Player_1', Player_1.pot) )
    
    # For subsequent games, remove bankrupt players
    L3 = copy.copy(initial_playerlist)
    for gamer in initial_playerlist:
        if gamer.pot <= 0:
            print (str(gamer.name) + " has gone bankrupt! ")
            L3.remove(gamer)
    initial_playerlist = L3
    playerlist = initial_playerlist # to re-add players who won BJ or went bust
    
    # print, for each gamer, their name, what's at stake and their pot
    maxbet=0
    for gamer in playerlist:
        if gamer.dealer==False:
            dummy2 = 0
            while dummy2 == 0 :
                try:
                    dosh = int(input(gamer.name + ' has ' + str(gamer.pot) + ' available; how much to play? '))
                    if dosh <= gamer.pot:
                        gamer.place_bet(dosh)
                        dummy2 = 1
                    else:
                        print ("Live within your means!")
                    if dosh < 0: # come 
                        print ("Don't be a douche.")
                except:
                    print (gamer.pot, gamer.betting_box) ############ something going wrong here
                    print ("Integer pleeze, geeze xxx @) ")
            print (gamer.name + ' bets ' + str(gamer.betting_box) + '; owns: ' + str(gamer.pot))
            if gamer.betting_box > maxbet:
                maxbet=gamer.betting_box
    Dealer.place_bet = maxbet                                                    ## see below
    print(Dealer.name + ' matches max bet of ' + str(Dealer.place_bet))       ### actually get rid; just deal with xchng at time




    ###### (5) Deal out cards by populating resp. 'hand' attributes of players-------------------------------------------------
    for gamer in playerlist:
        gamer.chuck()                                  # for debugging only????

        
    for gamer in playerlist*2 : # see next comment: ans is: x=input('name of deck: '), use x below
        # generalise as per previous comment
        gamer.receive(my_deck.take_1()) # prev line takes card, this line puts in looped gamer's hand

    #for gamer in playerlist:
        #if gamer.dealer==False:
            #print(gamer.name + ' holds ' + str(gamer.hand))
        #else:
            #print ("Dealer's face-up card is " + str(gamer.hand[1]))


    #(5.1) "graphics"
    def print_table():
        for gamer in playerlist:
            if gamer.dealer==False:
                print (gamer.name + ' owns: ' + str(gamer.pot) + '; at stake: ' + str(gamer.betting_box) + 
                       '; hand:' + str(gamer.hand) + '; value: ' + str(sum(gamer.hand)) )
            else:
                print ("Dealer's face-up card is " + str(gamer.hand[1]))

    from IPython.display import clear_output
    clear_output()
    print_table()



    # (6) loops for main play ------------------------------------------------------------------------------------------------
    #count=0------------------------------------------------------------------------------
    #while len(playerlist)>0 and count<len(playerlist)-1: #  no, not here - outermost loop for the evening

    def pause():
        programmePause = input('Press ENTER key to continue...')

    
    L2 = copy.copy(playerlist) # to avoid skipping when items are removed mid-loop
    for gamer in playerlist:
        if gamer.dealer==False: # just do the following for non-dealer players

            # Blackjack check
            if sum(gamer.hand) == 21:
                print (str(gamer.name) + ' has Blackjack!')
                if Dealer.hand == 21:
                    print (gamer.name + ' ties.') # Rare event of player and dealer having natural BJ
                    gamer.tie()
                else:
                    print (gamer.name + ' beats Dealer and wins at 3:2.')
                    Dealer.pot -= gamer.pot
                    gamer.win_BJ()
                    print (gamer.name + ' now has a pot of ' + str(gamer.pot))
                L2.remove(gamer)
                pause()

            # If no BlackJack, do Bust check; goes through hand lowering any Aces only as needed
            bustcount=0         # this used to be by inplay=True, 2 levels out
            inplay=True
            while inplay==True:


                while sum(gamer.hand)>21 and bustcount<len(gamer.hand): #NB if not over 21, code will skip this clause
                    for card in gamer.hand:
                        if card.low == False:
                            card.highlow() 
                            bustcount += 1
                if sum(gamer.hand)>21:
                    print (gamer.hand)
                    print (str(gamer.name) + ' is bust! ' + str(gamer.betting_box) + ' goes to Dealer.' )
                    pause()
                    L2.remove(gamer)
                    Dealer.pot += (gamer.betting_box)
                    gamer.lose()
                    print ('Remaining players: ' + str(L2))
                    inplay=False

                # If not bust, keep playing (bet and twist loop)
                else:
                    if gamer in L2:
                        clear_output()
                        print_table()
                        if input(gamer.name + ': Twist? (Y/N) ') in ['Y', 'y']:         #(1) if they 'twist'... (if not, skip)
                            gamer.place_bet(input(gamer.name + ': How much more to bet? '))    # place bets
                            gamer.receive(my_deck.take_1())                       #(2) give them a new card
                            print(gamer.hand)
                        else:
                            inplay=False
                    else:
                        inplay=False
    playerlist = L2 # the latter part of skipping-avoidance....... but excludes Blackjack winner
    clear_output()
    print_table()

    #count += 1
        #print (count)



    #(7) Finish off (dealer shows, follows algorithm, money is exchanged)---------------------------------------------------------

    #(7.1) update "graphics"
    def print_table_final():
        for gamer in playerlist:
            print (gamer.name + ' owns: ' + str(gamer.pot) + '; at stake: ' + str(gamer.betting_box) + 
                   '; hand:' + str(gamer.hand) + '; value: ' + str(sum(gamer.hand)) )

    #(7.2) action (loop through with comparisons - use dramatic pauses)
    if len(playerlist)>1: # sometimes all players go bust! (Dealer is in list)
        print ('Dealer reveals hidden card... ') 
        pause()
        clear_output()
        print_table_final()
        if sum(Dealer.hand)==21:          # If Dealer has Blackjack
            #come back to this
            print ('Dealer has Blackjack! Beats all non-BJs (even 21), ties with any BJs. ')
            for gamer in playerlist:
                if gamer.dealer==False:
                    if sum(gamer.hand)==21 and len(gamer.hand)==2:
                        gamer.tie()
                        print (gamer.name + ' ties.')
                    else:
                        gamer.lose()
                        print (gamer.name + ' loses.')
        else:#test1
            dealercontinue=1
            while sum(Dealer.hand)<17 or sum(Dealer.hand)>21 and dealercontinue==1:
                print ("Dealer must receive...")
                pause()
                Dealer.receive(my_deck.take_1()) # dealer receives
                dealerbust=1
                while sum(Dealer.hand)>21 and dealerbust<len(Dealer.hand): # if new cards busts, do Aces check/ lower them
                    for card in Dealer.hand:
                        if card.low==False:
                            card.highlow()
                        dealerbust += 1
                if sum(Dealer.hand)>21: # if still bust, exit
                    print ('Dealer loses! All remaining players win.')
                    for gamer in playerlist:
                        if gamer.dealer==False:
                            Dealer.pot -= gamer.pot
                            gamer.win() 
                    dealercontinue=0
                clear_output()
                print_table_final()

            # now dealer must stop dealing coz sum 17 or higher
            # Aces low check then Bust routine
            dealerbust=0
            while sum(Dealer.hand)>21 and dealerbust<len(Dealer.hand):
                for card in Dealer.hand:
                    if card.low==False:
                        card.highlow()
                    dealerbust += 1
                clear_output()
                print_table_final()
                
            if sum(Dealer.hand)>21:
                # come back to this
                print ('Dealer loses! All remaining players win.')
                for gamer in playerlist:
                    if gamer.dealer==False:
                        Dealer.pot -= gamer.betting_box
                        gamer.win()                
            else: 
                for gamer in playerlist: # loop through non-dealer players checking sum vs dealer's
                    if gamer.dealer==False:
                        if sum(gamer.hand) > sum(Dealer.hand):
                            Dealer.pot -= gamer.betting_box
                            gamer.win()
                            print (gamer.name + ' wins with ' + str(sum(gamer.hand)) + " vs Dealer's " + str(sum(Dealer.hand)))
                        elif sum(gamer.hand) == sum(Dealer.hand):
                            gamer.tie()
                            print (gamer.name + ' ties with ' + str(sum(gamer.hand)) + " vs Dealer's " + str(sum(Dealer.hand)))
                        else:
                            Dealer.pot += gamer.betting_box
                            gamer.lose()
                            print (gamer.name + ' loses with ' + str(sum(gamer.hand)) + " vs Dealer's " + str(sum(Dealer.hand)))


    print_table_final()
    pause()
    for gamer in initial_playerlist:
        print (gamer.name + ' owns: ' + str(gamer.pot)  )
        
    if input('Again? ') not in ['Y', 'y', 'Yes', 'yes'] or len(initial_playerlist)<2:
        keepplaying = 0
        print ("Thank you, come again.")
    else:
        clear_output()
            # If insufficient cards, regenerate the deck
        print ("Remaining cards: " + str(len(my_deck.cards)))
        if len(my_deck.cards) < 11*len(playerlist):
            #my_deck.cards=[] # Can I do this?
            my_deck = Deck(decksize=numdecks)


Jesse owns: 900; at stake: 160; hand:[3 of Clubs, 4 of Spades, 5 of Clubs, 5 of Clubs, 2 of Spades]; value: 19
Dealer owns: 1000060; at stake: 0; hand:[6 of Diamonds, Jack of Diamonds, Ace of Spades]; value: 17
Jesse wins with 19 vs Dealer's 17
Jesse owns: 1220; at stake: 0; hand:[3 of Clubs, 4 of Spades, 5 of Clubs, 5 of Clubs, 2 of Spades]; value: 19
Dealer owns: 999900; at stake: 0; hand:[6 of Diamonds, Jack of Diamonds, Ace of Spades]; value: 17
Press ENTER key to continue...
Boli owns: 880
Jesse owns: 1220
Dealer owns: 999900


# 